In [4]:
import numpy as np
from keras.datasets import fashion_mnist as fmn
from keras.models import Sequential, Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.callbacks import EarlyStopping

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Problem 1
Build a CNN comparable in structure to the one in the reading you did on CNNs, but train and test it on the fashion-MNIST dataset.  Adjust some of the parameters and compare the results.  You should be able to get performance better than any of the classifiers we have used on Fashion-MNIST so far.

In [8]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.3 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [9]:
(X_train, y_train), (X_test, y_test) = fmn.load_data() # fetch CIFAR-10 data

In [10]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
num_train, height, width, depth = X_train.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_test) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [11]:
inp = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
early_stopping = EarlyStopping(monitor='loss', patience=4)
model.fit(X_train, Y_train,                # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1,
          callbacks=[early_stopping]) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1)  # Evaluate the trained model on the test set!

Train on 54000 samples, validate on 6000 samples
Epoch 1/200
54000/54000 [==============================] - 141s 3ms/step - loss: 0.4530 - acc: 0.8351 - val_loss: 0.2791 - val_acc: 0.8965
Epoch 2/200
54000/54000 [==============================] - 131s 2ms/step - loss: 0.2946 - acc: 0.8924 - val_loss: 0.2456 - val_acc: 0.9065
Epoch 3/200
54000/54000 [==============================] - 130s 2ms/step - loss: 0.2584 - acc: 0.9050 - val_loss: 0.2293 - val_acc: 0.9175
Epoch 4/200
54000/54000 [==============================] - 127s 2ms/step - loss: 0.2351 - acc: 0.9128 - val_loss: 0.2143 - val_acc: 0.9235
Epoch 5/200
54000/54000 [==============================] - 127s 2ms/step - loss: 0.2168 - acc: 0.9210 - val_loss: 0.2066 - val_acc: 0.9238
Epoch 6/200
54000/54000 [==============================] - 133s 2ms/step - loss: 0.2072 - acc: 0.9246 - val_loss: 0.2172 - val_acc: 0.9198
Epoch 7/200
54000/54000 [==============================] - 131s 2ms/step - loss: 0.1956 - acc: 0.9278 - val_loss: 0.1

[0.24831310351341962, 0.9309]

This is indeed better than the previous neural nets that we have done. <br>
XGBoost got 88.9% accuracy.<br>
My first neural net assignment did very poorly. (~17% and ~40%) <br>
My second neural net assignent clocked out just below 90% like XGBoost did.